# Calibration of motor current

In [ ]:
%matplotlib notebook

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd

from tools.data_loader import moving_average

import glob
import os

In [ ]:
class Motor:
    
    def __init__(self, motorNumber):
        
        self.motorNumber = motorNumber
        self.currentESC = self.get_ESC_current(motorNumber)
        self.voltageESC = self.get_ESC_voltage(motorNumber)
        self.voltageMultimeter = self.get_multimeter_voltage(motorNumber)
        self.currentMultimeter = self.get_multimeter_current(motorNumber)
        
        self.timeCurrent = ((self.currentESC['t'].values - min(self.currentESC['t'].values)) / 10e5)
        self.timeVoltage = ((self.voltageESC['t'].values - min(self.voltageESC['t'].values)) / 10e5)

        self.omegaVoltage = self.voltageESC[f'omega{motorNumber}'].values
        self.omegaCurrent = self.currentESC[f'omega{motorNumber}'].values

        self.voltage = self.voltageESC[f'U1{motorNumber}'].values
        self.current = self.currentESC[f'I1{motorNumber}'].values
        
        self.voltageSequences = self.get_voltage_sequence()
        self.currentSequences = self.get_current_sequence()
    
    
    @staticmethod
    def get_files_for_limit(limit, flight, discard=True):
    
        v_induced = 14.345219306215128
        v_limit = limit * 0.01 * v_induced
        data_directory = flight + f'hover_{limit}_limit'
        csv_files = glob.glob(os.path.join(data_directory, "*.csv"))
        sorted_files = []

        # sort sequences in ascending order
        for i in range(len(csv_files)):
            for e in csv_files:
                if f'sequence_{i}_' in e:
                    #print(e)
                    sorted_files.append(e)
                    break

        # discard first and land sequence (vehicle on ground before takeoff and after landing)
        if discard:
            return sorted_files[1:-1]

        else:
            return sorted_files
    
    
    def get_ESC_current(self, motorNumber):
        
        filepath = f'/home/gregor/Masterarbeit/flight_data/2022-08-08_test_rig/current_motor_{motorNumber}/' 
        filesCurrent = self.get_files_for_limit(10, filepath, discard=False)
        
        currentData = pd.DataFrame()
        
        for file in filesCurrent:
            currentData = pd.concat([currentData, pd.read_csv(file)])
        
        return currentData
    
    
    
    def get_ESC_voltage(self, motorNumber):
        
        filepath = f'/home/gregor/Masterarbeit/flight_data/2022-08-08_test_rig/voltage_motor_{motorNumber}/'
        filesVoltage = self.get_files_for_limit(10, filepath, discard=False)
        
        voltageData = pd.DataFrame()
        
        for file in filesVoltage:
            voltageData = pd.concat([voltageData, pd.read_csv(file)])
        
        return voltageData
    
    
    @staticmethod
    def get_multimeter_voltage(motorNumber):
        
        filepath = f'/home/gregor/Masterarbeit/flight_data/2022-08-08_test_rig/voltage_data.csv'
        voltage_measurements = pd.read_csv(filepath)
        
        return voltage_measurements.iloc[motorNumber-1].values

    
    @staticmethod
    def get_multimeter_current(motorNumber):
        
        folder = f'/home/gregor/Masterarbeit/flight_data/2022-08-08_test_rig/current_data_multimeter/'
        file = f'current_motor{motorNumber}.csv'
        
        current_measurement = pd.read_csv(folder+file)
        
        return current_measurement.iloc[0].values
    
    
    def get_current_sequence(self):
        
        relevantRpmValues = np.unique(self.omegaCurrent[self.omegaCurrent>=1000], return_counts=True) 
        frequentRpmValues = relevantRpmValues[0][relevantRpmValues[1] > 20]
        filteredIndices = np.nonzero(np.isin(self.omegaCurrent, frequentRpmValues)==1)[0]
        
        startMask = np.where(np.diff(filteredIndices,prepend=0)>10) # search for the gaps between the sequences
        sequenceStart = filteredIndices[startMask]

        endMask = np.where(np.abs(np.diff(np.flip(filteredIndices),prepend=0))>10)
        sequenceEnd = np.flip(np.flip(filteredIndices)[endMask])
        
        filteredCurrentSequence = []
        
        for i in range(sequenceStart.size):
    
            intersection = np.intersect1d(np.arange(sequenceStart[i],sequenceEnd[i]), filteredIndices)
            
            # remove first and last 5 values to get rid of outliers
            if not intersection.size == 0:
                filteredCurrentSequence.append(intersection[5:-5])
            
        return filteredCurrentSequence
    
    
    def get_voltage_sequence(self):
        
        relevantRpmValues = np.unique(self.omegaVoltage[self.omegaVoltage>=1000], return_counts=True) 
        frequentRpmValues = relevantRpmValues[0][relevantRpmValues[1] > 20]
        filteredIndices = np.nonzero(np.isin(self.omegaVoltage, frequentRpmValues)==1)[0]
        
        startMask = np.where(np.diff(filteredIndices,prepend=0)>10) # search for the gaps between the sequences
        sequenceStart = filteredIndices[startMask]

        endMask = np.where(np.abs(np.diff(np.flip(filteredIndices),prepend=0))>10)
        sequenceEnd = np.flip(np.flip(filteredIndices)[endMask])
        
        filteredVoltageSequence = []
        
        for i in range(sequenceStart.size):
    
            intersection = np.intersect1d(np.arange(sequenceStart[i],sequenceEnd[i]), filteredIndices)
            
            # remove first and last 5 values to get rid of outliers
            if not intersection.size == 0:
                filteredVoltageSequence.append(intersection[5:-5])
            
        return filteredVoltageSequence

In [ ]:
# get data for all 8 motors

motor = []
for i in range(1,9):
    motor.append(Motor(i))

In [ ]:
# Show voltage measurements for all motors

n_motors = 8
fig0, ax0 = plt.subplots(4,2, figsize=(10,16))

# raw data
for i in range(4):
    ax0[i][0].scatter(motor[i].timeVoltage, motor[i].omegaVoltage, color='lightgray', label='RPM raw')
    ax0[i][1].scatter(motor[i].timeVoltage, motor[i].voltage, color='lightgray', label='esc-voltage raw')

    # plot the sequences 

    for j, seq in enumerate(motor[i].voltageSequences):

        time = motor[i].timeVoltage[seq]
        omega =  motor[i].omegaVoltage[seq]
        voltage = motor[i].voltage[seq]
        t = (max(time) + min(time))/2
        mean_voltage = np.mean(voltage)

        if(j==0):
            ax0[i][0].scatter(time, omega, color='deepskyblue', label = 'RPM filtered')
            ax0[i][1].scatter(time, voltage, color='deepskyblue', label='esc-voltage filtered')
            ax0[i][1].scatter(t, mean_voltage, s=50, marker='x', color='b', label='esc-voltage mean')
            ax0[i][1].scatter(t, motor[i].voltageMultimeter[j], s=50, marker='x', color='r',label='multimeter-voltage')
        else:
            ax0[i][0].scatter(time, omega, color='deepskyblue')
            ax0[i][1].scatter(time, voltage, color='deepskyblue')
            ax0[i][1].scatter(t, mean_voltage, s=50, marker='x', color='b')
            ax0[i][1].scatter(t, motor[i].voltageMultimeter[j], s=50, marker='x', color='r')
    
    ax0[i][0].set_xlabel('Time [s]')
    ax0[i][0].set_ylabel('$RPM$')
    ax0[i][0].set_ylim(0,4500)
    ax0[i][0].grid()
    ax0[i][0].legend()

    ax0[i][1].set_xlabel('Time [s]')
    ax0[i][1].set_ylabel('Voltage')
    ax0[i][1].set_ylim(25,35)
    ax0[i][1].grid()
    ax0[i][1].legend()

    ax0[i][0].set_title(f'Motor {i+1}: RPM over time')
    ax0[i][1].set_title(f'Motor {i+1}: Voltage over time')

fig0.suptitle('Voltage measurements', y=0.9925, fontsize=16)

fig0.tight_layout()

In [ ]:
# Show current measurements for all motors

fig1, ax1 = plt.subplots(4, 2, figsize=(10,16))


# raw data
for i in range(4):
    ax1[i][0].scatter(motor[i].timeCurrent, motor[i].omegaCurrent, color='lightgray', label='RPM raw')
    ax1[i][1].scatter(motor[i].timeCurrent, motor[i].current, color='lightgray', label='esc-voltage raw')

    # plot the sequences 

    for j, seq in enumerate(motor[i].currentSequences):

        time = motor[i].timeCurrent[seq]
        omega =  motor[i].omegaCurrent[seq]
        current = motor[i].current[seq]
        t = (max(time) + min(time))/2
        mean_current = np.mean(current)

        if(j==0):
            ax1[i][0].scatter(time, omega, color='deepskyblue', label = 'RPM filtered')
            ax1[i][1].scatter(time, current, color='deepskyblue', label='esc-voltage filtered')
            ax1[i][1].scatter(t, mean_current, s=50, marker='x', color='b', label='esc-voltage mean')
            ax1[i][1].scatter(t, motor[i].currentMultimeter[j], s=50, marker='x', color='r',label='multimeter-voltage')
        else:
            ax1[i][0].scatter(time, omega, color='deepskyblue')
            ax1[i][1].scatter(time, current, color='deepskyblue')
            ax1[i][1].scatter(t, mean_current, s=50, marker='x', color='b')
            ax1[i][1].scatter(t, motor[i].currentMultimeter[j], s=50, marker='x', color='r')
    
    ax1[i][0].set_xlabel('Time [s]')
    ax1[i][0].set_ylabel('$RPM$')
    ax1[i][0].set_ylim(0,4500)
    ax1[i][0].grid()
    ax1[i][0].legend()

    ax1[i][1].set_xlabel('Time [s]')
    ax1[i][1].set_ylabel('Current')
    #ax1[i][1].set_ylim(25,35)
    ax1[i][1].grid()
    ax1[i][1].legend()

    #ax0[i][0].set_ylim(0,4500)
    #ax0[1][0].set_ylim(0,4500)

    ax1[i][0].set_title(f'Motor {i+1}: RPM over time')
    ax1[i][1].set_title(f'Motor {i+1}: Current over time')
    #ax0[1][0].set_title(f'RPM over time')
    #ax0[1][1].set_title(f'Current over time')

    
fig1.suptitle('Current measurements', y=0.9925, fontsize=16)

fig1.tight_layout()